# # Rendering Plant Integrations for MiFlora Sensors from CSV

This notebook will focus on the generating the Plant Integration specifically for the Miflora Plant Sensors that were used with the ESPHome Configuration Generation in this [project](./AutoGenerate%20ESP32%20Configurations.ipynb)

Date of Project: July 24 2022
Software and Devices Used: 
- Home Assistant 2022.7
- Miflora Xiaomi BLE Plant Sensor
- ESP32-Wroom-RevD

Further notebooks will be created to document the other portions of this project which will be available on my Github repositoriy located here.

https://github.com/netmanchris/HassPlants

The Goal of this project will be to do the following

- Filter out only the plants in the plants_worksheet.csv file which are listed as having MiFlora sensors. 

- Plant Definitions - One per room. This should automatically generate the plant definition including the appropriate binding to the right sensor entity for the specific MiFlora sensors.

```
{% for plant in plants %}{% if plant.sensor_model == 'Miflora' %}
{{plant.name}}:
    sensors:
      moisture: sensor.plant_{{plant.sensor}}_moisture
      temperature: sensor.plant_{{plant.sensor}}_temperature
      conductivity: sensor.plant_{{plant.sensor}}_soil_conductivity
      brightness: sensor.plant_{{plant.sensor}}_illuminance{% for plant_type in plant_types%}{% if plant.plant_type == plant_type.species%}
    min_moisture: {{plant_type.min_moisture}}
    max_moisture: {{plant_type.max_moisture}}
    min_conductivity: {{plant_type.min_conductivity}}
    max_conductivity: {{plant_type.max_conductivity}}
    min_temperature: {{plant_type.min_temperature}}
    max_temperature: {{plant_type.max_temperature}}
    #min_brightness: {{plant_type.min_brightness}}
    max_brightness: {{plant_type.max_brightness}}{% endif%}{% endfor %}
    check_days: 3

    {% endif %}{% endfor %}
```




# Generating the Configuration Files

In this section, we'll go through the python code used to generate the actual configurations




In [3]:
import pandas as pd
from jinja2 import Template
import os

# Load Required Data Files CSV

In this section, we will load the plants CSV file which will be used to create the ESPHome configurations to bind the Miflora BLE Plant sensors to a specific ESP32. 

There are two files needed here.

1. plants_worksheet.csv which includes the various plants, sensors, room locations, corresponding script to flash lights.

2. plants_attributes.csv which includes the specific min/max values for the various sensors to be used with the plant integration








In [4]:
#plants_df = pd.read_csv('data/plants_worksheet.csv', index_col=0)
plants_df = pd.read_csv('data/plants_worksheet_sample.csv', index_col=0)
plant_attribs = pd.read_csv('data/plant_attrributes.csv', index_col=0).to_dict('records')

## Create Rooms List

In this step, we will create a list of the room names defined in the CSV file. Defining the room names logically is important in that ESP32 configurations, Plant entity definitions, and dashboards for each individual room will be automatically generated through this code. Spending some time thinking about how you want to organize your plants dashboards is a very, very good idea.

Note: I've got logic in the jinja2 files that will check to see whether the element in the room list is of type "string" which will allow sensors unassigned to rooms to be ignored. 

TODO - CHECK TO MAKE SURE THIS IS TRUE FOR ALL TEMPLATES

In [7]:
rooms_list = list(plants_df.room.unique())
rooms_list

['boys_bedroom',
 'dining_room',
 'downstairs_bathroom',
 'games_room',
 'home_office',
 'kitchen',
 'living_room',
 'master_bedroom',
 nan]

## Loading the Plants Integration Jinja2 Templates

In this step, we load the plants specific templates from the templates folder and load it as a jinja2 template.



In [9]:
with open('templates/room_plants.j2') as f:
    room_template = f.read()
j2_plantroom_template = Template(room_template)

## Rendering Plant Configurations

We will now render the Plant Configurations where one file will be generated for each room




Note: As a prerequisite, the directory structure has to preexist to be able to properly save the file configurations to disk. 

Make sure that the folder "plants" exists under the output folder.






In [11]:
# Includes the manual plants
# Includes the LilyGo plants

for room in rooms_list:
    if type(room) is str:  
        room = [room]
        room_df = plants_df[plants_df.room.isin(room)]
        #print (len(room_df))
        if len(room_df) > 0:
            #print (room)
            #room_df = room_df[room_df.sensor_model.eq('Miflora')]
            room_dict = room_df.to_dict('records')
            #print (len(room_dict))

            # OUTPUT A SINGLE FILE FOR EACH ROOM WITH ALL THE MiFLORA PLANTS INTEGRATION YAML WITH THE PROPER MIFLORA SENSOR NAMES
            with open('output/plants/'+room_dict[0]['room']+'.txt', 'w') as f: 
                f.write(j2_plantroom_template.render(plants=room_dict, plant_types=plant_attribs))

# Sample Output

The code above should generate a single file per room which contains all of the required plants.

The following is the a sample of the file labeled "dining_room.txt".

As you can see, the plant definition is automatically using the sensor name which matches that auto-generated in the [Auto Generation of ESP32 Configurations for use with ESP Home](./AutoGenerate%20ESP32%20Configurations.ipynb)section of this project

```
DR_Dracaena_1:
    sensors:
      moisture: sensor.plant_s7_moisture
      temperature: sensor.plant_s7_temperature
      conductivity: sensor.plant_s7_soil_conductivity
      brightness: sensor.plant_s7_illuminance
    min_moisture: 15
    max_moisture: 60
    min_conductivity: 200
    max_conductivity: 1500
    min_temperature: 10
    max_temperature: 35
    #min_brightness: 2000
    max_brightness: 50000
    check_days: 3

```

The configuration should be cut and paste into a file labeled %room_name%.yaml in the appropriate folder

```
config
   ->plants
```
       
This should also require adding a line to the configuration.yaml file to make sure that the structure is included in the processing of the overall configuration.


```plant: !include_dir_merge_named plants```

![image](./images/config_plants_yaml.png)


Once you reload your configuration or restart your home assistant instance, you should be able to verify that the plants are shown as follows


Navigate to **Settings > Devices & Services > entities** to validate that the plants were created as desired.

![image](./images/hass_plant_entities.png)

By click on a specific entity you can validate the current status and configuration 

![image](./images/plant_entity_1.png)

By clickig on the attributes link, you can also validate the specific sensors that any individual plant.

![image](./images/plant_entity_2.png)


# End Goal

The entire purpose of this is to bind a specific setting to a specific plant so that you are now able to use this device in your home assistant dashboards.

Now that you have bound all the sensors to a specific plant, you'll be able to use the plant card to show the current status of any individual plant


![image](./images/plant_card_1.png)